In [ ]:
%pip install requests 
%pip install beautifulsoup4

In [ ]:
import requests
from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup
import urllib.parse

In [ ]:
def get_first_file(year, doy):
    u = (
        f"https://gpm1.gesdisc.eosdis.nasa.gov/opendap/"
        f"GPM_L2/GPM_2AKu.07/{year}/{doy:03d}/contents.html"
    )
    r = requests.get(u)
    if not r.ok:
        return None
    s = BeautifulSoup(r.text, "html.parser")
    for a in s.find_all("a"):
        h = a.get("href", "")
        if h.endswith(".HDF5.dmr.html"):
            return h.replace(".dmr.html", "")
    return None

In [ ]:
def get_first_hdf5(url: str) -> str:
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    for link in soup.find_all("a", href=True):
        if link["href"].endswith(".HDF5"):
            return urllib.parse.urljoin(url, link["href"])
    return None

In [ ]:
def download_file(u, f):
    r = requests.get(u)
    r.raise_for_status()
    with open(os.path.join("data", f), "wb") as file:
        file.write(r.content)

In [ ]:
b = "https://gpm1.gesdisc.eosdis.nasa.gov/opendap/GPM_L2/GPM_2AKu.07"
t = "?dap4.ce=/FS_Longitude[0:1:20][0:1:20];/FS_Latitude[0:1:20][0:1:20];/FS_VER_airTemperature[0:1:20][0:1:20][0:1:20];/FS_PRE_flagPrecip[0:1:20][0:1:20];/FS_SLV_precipWater[0:1:20][0:1:20][0:1:20];/FS_SLV_precipWaterIntegrated[0:1:20][0:1:20][0:1:1];/FS_ScanTime_Year[0:1:20];/FS_ScanTime_Month[0:1:20]"
os.makedirs("data", exist_ok=True)

for y in [2016, 2017, 2018]:
    for i in range(20):
        f = get_first_file(y, i)
        if f:
            nc4_filename = f"{f}.dap.nc4"
            file_path = os.path.join("data", nc4_filename)
            if os.path.exists(file_path):
                print(f"File {nc4_filename} already exists. Skipping download.")
                continue
            u = f"{b}/{y}/{i:03d}/{f}.dap.nc4{t}"
            print(f"Downloading {nc4_filename}...")
            download_file(u, nc4_filename)